<a href="https://colab.research.google.com/github/Benedictakel/Stock_Price_Trend_Prediction/blob/main/Stock_Price_Trend_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [ ]:
# Download Apple stock data as an example
data = yf.download("AAPL", start="2020-01-01", end="2024-12-31")

data.head()


In [ ]:
# Create target column: 1 if next day's Close > today's Close, else 0
data['Target'] = (data['Close'].shift(-1) > data['Close']).astype(int)

# Drop last row with NaN target
data.dropna(inplace=True)


In [ ]:
# Use moving averages and daily returns as features
data['MA5'] = data['Close'].rolling(window=5).mean()
data['MA10'] = data['Close'].rolling(window=10).mean()
data['MA20'] = data['Close'].rolling(window=20).mean()
data['Return'] = data['Close'].pct_change()
data['Volatility'] = data['Return'].rolling(5).std()

# Drop rows with NaNs (from rolling calculations)
data.dropna(inplace=True)


In [ ]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA5', 'MA10', 'MA20', 'Return', 'Volatility']
X = data[features]
y = data['Target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)


In [ ]:
y_pred = model.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt="d", cmap="Greens")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
latest_data = X.tail(1)
latest_scaled = scaler.transform(latest_data)
next_day_trend = model.predict(latest_scaled)

print("Prediction for Next Day:", "🔼 Up" if next_day_trend[0] == 1 else "🔽 Down")
